In [63]:
from pyclustering.cluster.clarans import clarans;
from pyclustering.utils import timedcall;
from sklearn import datasets
import numpy as np
import pandas as pd
import operator

df = pd.read_csv('Rentabilidad de estrategias base 100.csv')

def clara(df,k=3,v=4):
    df['year'] = df['Date'].apply(lambda x: x.split('-')[0])
    df2 = df[df['year'].isin(['2017','2018','2019'])]
    df2 = df2.drop('year',axis=1)
    df2=df2.set_index('Date').tail(365)
    df3 = df2.tail(365).transpose()
    data = np.array(df3)

    data = data.tolist()

    #get a glimpse of dataset
    #print("A peek into the dataset : ",data[:4])


    """!
    @brief Constructor of clustering algorithm CLARANS.
    @details The higher the value of maxneighbor, the closer is CLARANS to K-Medoids, and the longer is each search of a local minima.
    @param[in] data: Input data that is presented as list of points (objects), each point should be represented by list or tuple.
    @param[in] number_clusters: amount of clusters that should be allocated.
    @param[in] numlocal: the number of local minima obtained (amount of iterations for solving the problem).
    @param[in] maxneighbor: the maximum number of neighbors examined.        
    """
    clarans_instance = clarans(data, k, 6, v);

    #calls the clarans method 'process' to implement the algortihm
    (ticks, result) = timedcall(clarans_instance.process);
    print("Execution time : ", ticks, "\n");

    #returns the clusters 
    clusters = clarans_instance.get_clusters();

    #returns the mediods 
    medoids = clarans_instance.get_medoids();

    dic = {}
    for var in range(len(clusters)):
        for var2 in clusters[var]:
            dic[var2] = var


    resultado = sorted(dic.items(), key=operator.itemgetter(0))

    dic = {}
    for var in resultado:
        dic[df3.index[var[0]]] = var[1]

    clf = pd.DataFrame.from_dict(dic,orient='index')
    clf = clf.reset_index()
    results = clf.rename(columns={'index':'strategies',0:'clusters'})

    sharpeclusters =[] 
    Features = df2
    for i in range(results['clusters'].nunique()):

        l = results.loc[results['clusters'] == i].index.values.astype(int).tolist()
        dfexp=Features.pct_change().iloc[: , l].sum(axis = 1, skipna = True)
        r = dfexp.mean()
        s = dfexp.std()

        sharpeclusters.append(r/s)
    sharpeclusters = np.asanyarray(sharpeclusters)

    selection = sharpeclusters.argmax()
    cond = results['clusters'] == selection

    results['selection'] = np.where(cond, 1, 0)

    return list(results['selection'])


In [65]:
clara(df,k=3,v=5)

Execution time :  6.946293000000001 



[1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]